In [ ]:
%%html
<h1 align=center>Caderno de Análise dos Dados do SPED - EFD Contribuições </h1>

In [ ]:
# Importar as bibliotecas para a realização da Análise ((não alterar o código))

from sqlalchemy import create_engine
import pymysql
import datetime as dt
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
# Conexão ao banco de dados do EFD-Cotribuições (não alterar o código)
conn = pymysql.connect(host="localhost",
                      port=3338,
                      user="spedfiscal",
                      passwd="spedfiscal",
                      database = "master",
                      charset='utf8')

In [ ]:
%%html
<h2 align="center"> Insira o CNPJ da entidade sem espaços, simbolos ou letras </h2>

In [ ]:
cnpj = int(input("Digite o número do CNPJ da entidade: "))

In [ ]:
%%html
<h4 align="center"> Copie na opção abaixo o nome do db para análise </h4>

In [ ]:
# Visualizando os nomes dos Bancos de Dados , entidades e CNPJ
pd.read_sql(f"select cnpj, nome_contribuinte, nomeBD, dataInicial FROM escrituracao WHERE cnpj = {cnpj} " , con=conn)

In [ ]:
conn.close()

In [ ]:
# Solicita o nome do banco de dados e instancia conn com o novo DB
db_name = input("Digite o nome do Banco de Dados: ")
conn = pymysql.connect(host="localhost",
                      port=3338,
                      user="spedfiscal",
                      passwd="spedfiscal",
                      database = db_name,
                      charset='utf8')

In [ ]:
%%html
<h2> Análise da Prestação de Serviços A100 e A170 </h2>

In [ ]:
# Carregando arquivos de serviços tabelas a100 e a170
reg_a170_servicos = pd.read_sql("select HASH, COD_ITEM, DESCR_COMPL, VL_ITEM, NAT_BC_CRED,COD_CTA FROM reg_a170 WHERE COD_ITEM != 'CRED'", con=conn)
reg_a170_servicos

In [ ]:
# merge entre as tabelas reg_a170 e a de contas contábeis reg_0500
reg_0500 = pd.read_sql("SELECT COD_CTA, NOME_CTA FROM reg_0500", con=conn)
reg_a170_servicos = pd.merge(reg_a170_servicos, reg_0500, on="COD_CTA")

In [ ]:
reg_a170_servicos

In [ ]:
# agrupando dados de soma e média dos valores
reg_a170_group_sum = reg_a170_servicos[["DESCR_COMPL","NOME_CTA","VL_ITEM"]].groupby(["DESCR_COMPL", "NOME_CTA"]).agg(["sum", "mean", "std", "min", "max"])
reg_a170_group_sum

In [ ]:
# Plotando os lançamentos dos registros A170 para cada descrição de atividade

plt.figure(figsize=(10,6))
plt.title("Registros A100 e A170 - Serviços")
sns.barplot(data=reg_a170_servicos, y="DESCR_COMPL", x="VL_ITEM", hue="NOME_CTA")
plt.show()

In [ ]:
# Box Plot das Receitas classificadas por tipo de atividade

plt.figure(figsize=(10,6))
plt.title("Registros A100 e A170 - Serviços")
sns.boxplot(data=reg_a170_servicos, x="DESCR_COMPL", y="VL_ITEM", hue="NOME_CTA")
plt.show()

In [ ]:
# Tabela de itens reg_0200
reg_0200 = pd.read_sql("SELECT COD_ITEM, DESCR_ITEM FROM reg_0200", con=conn)
reg_0200

In [ ]:
# Merge entre os dataframes reg_a170 e reg_0200 (registros de itens)
reg_a170_merge_cod_item = pd.merge(reg_a170_servicos, reg_0200, on="COD_ITEM")
reg_a170_merge_cod_item

In [ ]:
# Estatistica Descritiva A100 ou A170 por Descrição de itens
reg_a170_merge_cod_item_group_sum = reg_a170_merge_cod_item[["DESCR_ITEM","NOME_CTA","VL_ITEM"]].groupby(["DESCR_ITEM", "NOME_CTA"]).agg(["sum", "mean", "std", "min", "max"])
reg_a170_merge_cod_item_group_sum

In [ ]:
plt.figure(figsize=(10,6))
plt.title("Registros A100 e A170 - Serviços")
sns.barplot(data=reg_a170_merge_cod_item, y="DESCR_ITEM", x="VL_ITEM", hue="NOME_CTA")
plt.show()

In [ ]:
%%html
<br><br><Br>
<h2> Registros de saida e venda de mercadorias C100 </h2>

In [ ]:
# Tabelas de venda de produtos ou saídas de mercadorias
reg_c100 = pd.read_sql("SELECT * FROM reg_c100 WHERE IND_OPER=1", con=conn)
reg_c100

In [ ]:
%%html
<br><br><Br>
<h2> Notas Fiscais consolidadas de Serviços de Telecomunicações D600 </h2>

In [ ]:
# Acesso às tabelas D600 - Serviços de Telecomunicações
reg_d600 = pd.read_sql("SELECT HASH, QTD_CONS, DT_DOC_INI, VL_DOC FROM reg_d600", con=conn)
reg_d600

In [ ]:
plt.figure(figsize=(10,6))
plt.title("Distribuição das receitas de telecomunicações D600")
plt.xlabel("Lançamentos em index")
plt.ylabel("Valores em R$")
sns.scatterplot(data=reg_d600, x=reg_d600.index, y="VL_DOC")
plt.show()

In [ ]:
# Boxplot da distribuição de Serviços de Telecomunicações
plt.figure(figsize=(10,6))
plt.title("Registros D600 - Telecomunicações")
plt.ylabel("Valores em R$")
sns.boxplot(data=reg_d600, y="VL_DOC")
plt.show()

In [ ]:
# Histograma de distribuição das receitas de telecomunicações no mês:
plt.figure(figsize=(10,6))
plt.title("Registros D600 - Telecomunicações")
plt.xlabel("Valores Consolidados em R$")
plt.ylabel("Ocorrências")
sns.histplot(data=reg_d600, x="VL_DOC", hue="QTD_CONS")
plt.show()

In [ ]:
# Estatística descritiva das Receitas D600 de Telecomunicações
reg_d600_group_sum = round(reg_d600[["VL_DOC"]].agg(["sum", "mean", "std", "min", "max"]),2)
reg_d600_group_sum

In [ ]:
%%html
<h2> Registros de saída de outras atividades econômicas F100 </h2>

In [ ]:
# Acesso à tabela outros serviços F100 e merge com a tabela reg_0500
reg_f100 = pd.read_sql("SELECT HASH, COD_ITEM, DT_OPER, VL_OPER, DESC_DOC_OPER, COD_CTA FROM reg_f100", con=conn)
reg_f100 = pd.merge(reg_f100, reg_0500, on="COD_CTA")

In [ ]:
reg_f100

In [ ]:
# Operações ou lançamentos por descrição das operações

plt.figure(figsize=(10,6))
plt.title("Registros f100 - outras receitas")
plt.xlabel("Descrição da Atividade")
plt.ylabel("Valores de Receitas em R$")
sns.barplot(data=reg_f100, x="DESC_DOC_OPER", y="VL_OPER", hue="NOME_CTA")
plt.show()

In [ ]:
# Estatística descritiva
reg_f100_group_sum = round(reg_f100[["DESC_DOC_OPER", "NOME_CTA" ,"VL_OPER"]].groupby(["DESC_DOC_OPER", "NOME_CTA"]).agg(["sum", "mean", "std", "min", "max"]),2)
reg_f100_group_sum

In [ ]:
conn.close()

In [ ]:
%%html
<h3 align="center"> Execute a célula abaixo Se e somente Se desejar gravar as tabelas no formato excell </h3>

In [ ]:
# Importing files to Excell
month = input("Escreva o nome do mês em minúsculo, sem separações ou símbolos: ")
reg_a170_servicos.to_excel(f"servicos_a100_{month}.xlsx")
reg_a170_group_sum.to_excel(f"servicos_a100_soma_{month}.xlsx")
reg_d600.to_excel(f"servicos_telecom_d600_{month}.xlsx")
reg_f100.to_excel(f"outros_servicos_f100_{month}.xlsx")
reg_c100.to_excel(f"registros_c100_{month}.xlsx")